In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
import nest_asyncio

nest_asyncio.apply()
_ = load_dotenv(find_dotenv())

## Hello World

In [3]:
from pydantic_ai import Agent

agent = Agent(  
    'google-gla:gemini-1.5-flash',
    system_prompt='Be concise, reply with one sentence.',  
)

result = agent.run_sync('Where does "hello world" come from?')  
print(result.output)

"Hello, world!" originated as a sample program in Brian Kernighan's 1972 tutorial on the programming language B.



## Single Agent with Tools

In [ ]:
from dataclasses import dataclass
from pydantic import BaseModel, Field
from pydantic_ai import Agent, RunContext

### Search Agent

In [ ]:
from pydantic_ai.common_tools.duckduckgo import duckduckgo_search_tool

search_agent = Agent(  
    'google-gla:gemini-1.5-flash',
    system_prompt='Search DuckDuckGo for the given query and return the results.',  
    tools = [duckduckgo_search_tool()]
)

result = search_agent.run_sync(
    "Where does 'hello world' come from?"
)

/Users/tituslim/Documents/Personal Learning Folder/Personal Projects/friendly-happiness/.venv/lib/python3.12/site-packages/pydantic_ai/common_tools/duckduckgo.py:68: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  DuckDuckGoSearchTool(client=duckduckgo_client or DDGS(), max_results=max_results).__call__,


In [68]:
result

AgentRunResult(output='The "Hello, world!" program\'s origins are generally traced back to Brian Kernighan\'s 1972 internal document, "A Tutorial Introduction to the Language B".  While not the very first instance of a similar program, it\'s considered the seminal example that popularized the phrase and the practice of using it as a introductory programming exercise.  Since then,  "Hello, world!" has become a ubiquitous and iconic example used in countless programming tutorials and textbooks.\n')

In [8]:
from IPython.display import display, Markdown

display(Markdown(result.output))

The "Hello, world!" program's origins are generally traced back to Brian Kernighan's 1972 internal document, "A Tutorial Introduction to the Language B".  While not the very first instance of a similar program, it's considered the seminal example that popularized the phrase and the practice of using it as a introductory programming exercise.  Since then,  "Hello, world!" has become a ubiquitous and iconic example used in countless programming tutorials and textbooks.


### Stock Agent

In [60]:
from typing import Any, Optional, Annotated
from tqdm.notebook import tqdm
import yfinance as yf

@dataclass
class Username:
    name: Annotated[str, "The user name seeking the input"]

def get_ticker_data(
    tickers: list[str], 
    start_date: Optional[Annotated[str, "Start date of interest. For example '2025-01-01'"]] = None,
    end_date: Optional[Annotated[str, "End date of interest. For example '2025-06-30'"]] = None
) -> list[dict[str, Any]]:
    """Use this tool to get stock ticker data from Yahoo Finance"""
    return yf.download(tickers, start=start_date, end=end_date).to_json()

In [65]:
stock_agent = Agent(
    'google-gla:gemini-2.5-flash',
    deps_type=Username,
    tools=[get_ticker_data],
    system_prompt = "You are a helpful assistant"
)

In [66]:
response = await stock_agent.run(
    "Can you analyze the ticker data for Apple and Microsoft from 1 Jun 2025 - 30 Jun 2025?",
    deps = Username("Titus")
)

/var/folders/zb/r15p7t_d62d8m2s0623s22gh0000gn/T/ipykernel_63958/3106942276.py:15: FutureWarning: YF.download() has changed argument auto_adjust default to True
  return yf.download(tickers, start=start_date, end=end_date).to_json()
[*********************100%***********************]  2 of 2 completed


In [67]:
display(Markdown(response.output))

Here is an analysis of the ticker data for Apple (AAPL) and Microsoft (MSFT) for June 2025, based on the provided data from June 2, 2025, to June 27, 2025:

**Apple (AAPL)**
*   **Opening Price (June 2, 2025):** $200.28
*   **Closing Price (June 27, 2025):** $201.08
*   **Highest Price (during the period):** $206.24
*   **Lowest Price (during the period):** $195.07
*   **Overall Change:** The price increased by $0.80.
*   **Average Daily Volume:** Approximately 53.03 million shares.

**Microsoft (MSFT)**
*   **Opening Price (June 2, 2025):** $457.14
*   **Closing Price (June 27, 2025):** $495.94
*   **Highest Price (during the period):** $499.30
*   **Lowest Price (during the period):** $456.89
*   **Overall Change:** The price increased by $38.80.
*   **Average Daily Volume:** Approximately 19.62 million shares.